In [1]:
import pandas as pd
df_Paris = pd.read_csv("http://data.insideairbnb.com/france/ile-de-france/paris/2022-09-09/visualisations/listings.csv")

In [2]:
df_Paris.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,23441,Charming at Montmartre & rue Lepic- mobility l...,91706,Elise,NaN,Buttes-Montmartre,48.88408,2.33478,Entire home/apt,70,30,84,2022-08-31,0.55,1,115,3,"Available with a mobility lease only (""bail mo..."
1,24260,Lovely place near Champs Elysees,98012,Emmanuel,NaN,Batignolles-Monceau,48.88090,2.30388,Entire home/apt,85,30,30,2017-08-11,0.20,2,319,0,"Available with a mobility lease only (""bail mo..."
2,130420,Charming Apartment 2BR in Paris 9e,641777,Yassine,NaN,Opéra,48.88073,2.34297,Entire home/apt,213,1,188,2022-09-06,1.38,1,301,32,7510900711502
3,26562,The Eiffel Tower as your neighbour,107514,Judith,NaN,Vaugirard,48.85357,2.29232,Entire home/apt,120,6,165,2019-10-12,1.10,1,3,0,7511500915333
4,132994,charmant studio dans le marais,653074,Victoire,NaN,Temple,48.86341,2.35837,Entire home/apt,90,365,35,2017-03-13,0.26,1,365,0,NaN
